<a href="https://colab.research.google.com/github/aabdysheva/collab-files/blob/main/GAN_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
from torch import device
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [10]:
!pip install kaggle

In [6]:
%cd /content/gdrive/MyDrive/datasets/

/content/gdrive/MyDrive/datasets


In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/datasets"

In [8]:
!kaggle datasets download -d spandan2/cats-faces-64x64-for-generative-models

cats-faces-64x64-for-generative-models.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!mkdir cats
!unzip -qq cats-faces-64x64-for-generative-models.zip -d /cats/

mkdir: cannot create directory ‘cats’: File exists


In [10]:
image_size = 64
batch_size = 128
latent_size= 256
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

train = ImageFolder('/cats', transform=tt.Compose([tt.Resize(image_size),
                                                  tt.CenterCrop(image_size),
                                                  tt.ToTensor(),
                                                  tt.Normalize(*stats)]))
 
train_dl = DataLoader(train, batch_size, shuffle=True, pin_memory=True) #For loading data into batches

In [11]:
generator = nn.Sequential(
    
    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # 3 x 64 x 64
)

In [12]:
discriminator = nn.Sequential(
    # 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=True),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=True),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=True),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # 512 x 4 x 4

    nn.Conv2d(512, 64 , kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(64, 1 , kernel_size=1, stride=1, padding=0, bias=False),

    # 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid())

In [15]:
summary(discriminator.to(device), (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,136
       BatchNorm2d-2           [-1, 64, 32, 32]             128
         LeakyReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 16, 16]         131,200
       BatchNorm2d-5          [-1, 128, 16, 16]             256
         LeakyReLU-6          [-1, 128, 16, 16]               0
            Conv2d-7            [-1, 256, 8, 8]         524,544
       BatchNorm2d-8            [-1, 256, 8, 8]             512
         LeakyReLU-9            [-1, 256, 8, 8]               0
           Conv2d-10            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-11            [-1, 512, 4, 4]           1,024
        LeakyReLU-12            [-1, 512, 4, 4]               0
           Conv2d-13             [-1, 64, 1, 1]         524,288
      BatchNorm2d-14             [-1, 6

In [16]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

In [17]:
sample_dir = '/generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors).to(device)
    fake_fname = 'image-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [18]:
def train_discriminator(real_images, opt_d):
    opt_d.zero_grad()

    real_preds = discriminator(real_images).to(device) 
    real_targets = torch.ones(real_images.size(0), 1).to(device) 
    real_loss = F.binary_cross_entropy(real_preds, real_targets) 
    real_score = torch.mean(real_preds).item()
    
    latent = torch.randn(batch_size, latent_size, 1, 1).to(device) 
    fake_images = generator(latent).to(device)  

    fake_targets = torch.zeros(fake_images.size(0), 1).to(device) 
    fake_preds = discriminator(fake_images).to(device)  
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)  
    fake_score = torch.mean(fake_preds).item()

    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score

In [19]:
def train_generator(opt_g):
    opt_g.zero_grad()
    latent = torch.randn(batch_size, latent_size, 1,1).to(device) 
    # print(latent.shape)
    fake_images = generator(latent).to(device) 
    
    preds = discriminator(fake_images).to(device) 
    targets = torch.ones(batch_size, 1).to(device)
    loss = F.binary_cross_entropy(preds, targets) 
    
    loss.backward()
    opt_g.step()
    
    return loss.item(),latent

In [22]:
def fit(epochs=10, lr=0.0002, start_idx=1):
    torch.cuda.empty_cache()
    
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
  
    opt_d = torch.optim.Adam(discriminator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            
            real_images = real_images.to(device)
            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            
            loss_g, latent = train_generator(opt_g)
            
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        print(f"[{epoch+1}/{epochs}], loss_g: {loss_g:.4f}, loss_d: {loss_g:.4f}, real_score: {real_score:.4f}, fake_score: {fake_score:.4f}")
    
        save_samples(epoch+start_idx, latent, show=False)
    
    return losses_g, losses_d, latent, fake_scores

In [23]:
!pip install accelerate
from accelerate import notebook_launcher
notebook_launcher(fit())

  0%|          | 0/247 [00:00<?, ?it/s]

[1/10], loss_g: 2.2569, loss_d: 2.2569, real_score: 0.4336, fake_score: 0.1267
Saving image-0001.png


  0%|          | 0/247 [00:00<?, ?it/s]

[2/10], loss_g: 1.8783, loss_d: 1.8783, real_score: 0.3209, fake_score: 0.1386
Saving image-0002.png


  0%|          | 0/247 [00:00<?, ?it/s]

...

  0%|          | 0/247 [00:00<?, ?it/s]

[9/10], loss_g: 2.8241, loss_d: 2.8241, real_score: 0.2719, fake_score: 0.0662
Saving image-0009.png


  0%|          | 0/247 [00:00<?, ?it/s]

[10/10], loss_g: 2.5494, loss_d: 2.5494, real_score: 0.2665, fake_score: 0.0762
Saving image-0010.png
Launching training on one GPU.


TypeError: ignored